In [1]:
import os
import json
from typing import List, Literal, Optional, Text, Union, Dict, Any
from IPython.display import Image, display
from langchain_core.runnables.graph_mermaid import MermaidDrawMethod
from agents.agents_modules.workflow import build_agent_workflow
from agents.dataloader import load_dataset_by_name, extract_example

/Users/chinonsoosuji/opt/anaconda3/envs/lang2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def save_result_to_json(state: dict, filename: str = "result.json", directory: str = "results") -> None:
    """
    Saves the given agent workflow state to a JSON file in a specified directory.

    Args:
        state (dict): The final state returned by the agent graph execution.
        filename (str): The name of the JSON file to save.
        directory (str): The directory where the file will be saved.

    Raises:
        IsADirectoryError: If the target file path is a directory.
    """

    file_path = os.path.join(directory, filename)
    os.makedirs(directory, exist_ok=True)

    if os.path.isdir(file_path):
        raise IsADirectoryError(f"Cannot write to '{file_path}' because it is a directory.")

    # Recursively convert objects to serializable types
    def make_serializable(obj):
        if isinstance(obj, list):
            return [make_serializable(x) for x in obj]
        elif hasattr(obj, "model_dump"):  # Pydantic BaseModel
            return obj.model_dump()
        elif isinstance(obj, dict):
            return {k: make_serializable(v) for k, v in obj.items()}
        else:
            return obj

    serializable_state = make_serializable(dict(state))

    with open(file_path, "w", encoding="utf-8") as f:
        json.dump(serializable_state, f, indent=4)

    print(f"Results saved to {file_path}")


In [ ]:
# Exclude turku_hockey and conversational_weather
# rotowire, totto, sportsett_basketball, webnlg, dart, mlb
name = "webnlg"  
num = -101 # Sample id

data = load_dataset_by_name(name)
sample = extract_example(name, data["train"][num])
data = sample.get('input', '')
ground_truth = sample.get('references', "")
target = sample.get('target', "")

query = f"""You are an agent designed to generate text from data for a data-to-text natural language generation. You can be provided data in the form of xml, table, meaning representations, graphs etc. 
Your task is to generate the appropriate text given the data information without omitting any field or adding extra information in essence called hallucination.
Here is the data generate text using table data:
{data}"""

print(f"Input: {data}")
print(f"Target: {target}")
print(f"Ground Truth: {ground_truth}")

Loading dataset: webnlg
Input: ['Acharya_Institute_of_Technology | city | Bangalore', 'Acharya_Institute_of_Technology | established | 2000', 'Acharya_Institute_of_Technology | state | Karnataka', 'Acharya_Institute_of_Technology | country | "India"', 'Acharya_Institute_of_Technology | numberOfPostgraduateStudents | 700', 'Acharya_Institute_of_Technology | campus | "In Soldevanahalli, Acharya Dr. Sarvapalli Radhakrishnan Road, Hessarghatta Main Road, Bangalore – 560090."', 'Acharya_Institute_of_Technology | affiliation | Visvesvaraya_Technological_University']
Target: In Soldevanahalli, Acharya Dr. Sarvapalli Radhakrishnan Road, Hessarghatta Main Road, Bangalore – 560090 is the location of the Acharya Institute of Technology in the state of Karnataka, India. The Institute which was established in the year 2000 has 700 postgraduate students and is affiliated with the Visvesvaraya Technological University.
Ground Truth: 


In [ ]:
provider = "openai" #ollama, openai, hf, aixplain
add_plan = False # True or False
process_flow = build_agent_workflow(provider=provider)
# display(Image(process_flow.get_graph(xray=True).draw_mermaid_png(draw_method=MermaidDrawMethod.PYPPETEER)))
display(Image(process_flow.get_graph(xray=True).draw_mermaid_png()))

In [6]:
initial_state = {
    "user_prompt": query,
    "raw_data": data,
    "history_of_steps": [],
    "final_response": "",
    "next_agent": "",
    "next_agent_payload": "",
    "current_step": 0,
    "iteration_count": 0,
    "max_iteration": 60,
}

state = process_flow.invoke(initial_state, config={"recursion_limit": initial_state["max_iteration"]})
prediction = state['final_response']

ORCHESTRATOR OUTPUT: Thought: The user has provided structured input data in a format that includes entity-relation triples. The task requires transforming this data into a fluent and accurate natural language description. The input includes specific details about Acharya Institute of Technology, such as its location, establishment year, student numbers, and affiliation. I will rephrase this information into coherent sentences while ensuring all details are included.

Worker: 'surface realization'
Worker Input: 
The input data to be transformed is as follows:
- "Acharya_Institute_of_Technology | city | Bangalore"
- "Acharya_Institute_of_Technology | established | 2000"
- "Acharya_Institute_of_Technology | state | Karnataka"
- "Acharya_Institute_of_Technology | country | India"
- "Acharya_Institute_of_Technology | numberOfPostgraduateStudents | 700"
- "Acharya_Institute_of_Technology | campus | In Soldevanahalli, Acharya Dr. Sarvapalli Radhakrishnan Road, Hessarghatta Main Road, Bangalo

KeyError: 'finalizer'

In [ ]:
save_result_to_json(state, filename=f"{name}_{num}.json")

Results saved to results/webnlg_-101.json


In [ ]:
prediction

'Acharya Institute of Technology, established in 2000, is affiliated with Visvesvaraya Technological University. It is located in Bangalore, Karnataka, India. The campus is situated in Soldevanahalli on Acharya Dr. Sarvapalli Radhakrishnan Road, Hessarghatta Main Road, Bangalore – 560090. Currently, there are 700 postgraduate students enrolled.'

In [ ]:
ground_truth

''

In [ ]:
target

'In Soldevanahalli, Acharya Dr. Sarvapalli Radhakrishnan Road, Hessarghatta Main Road, Bangalore – 560090 is the location of the Acharya Institute of Technology in the state of Karnataka, India. The Institute which was established in the year 2000 has 700 postgraduate students and is affiliated with the Visvesvaraya Technological University.'